In [1]:
!pip install easyocr
!pip install opencv-python
!pip install torch torchvision numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 908.3/908.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.3/281.3 kB 7.1 MB/s eta 0:00:00


In [2]:
import easyocr
import cv2
import time
import numpy as np
from google.colab.patches import cv2_imshow
from concurrent.futures import ThreadPoolExecutor

# Initialize EasyOCR reader for GPU and CPU
gpu_reader = easyocr.Reader(['en'], gpu=True)
cpu_reader = easyocr.Reader(['en'], gpu=False)

video_file_path = "video2.mp4"  # Update with your actual video path

# Define different resolutions to test
resolutions = [(640, 480), (800, 600), (1024, 768), (1280, 720)]  # Add or modify resolutions as needed

# Function to process a single frame with both GPU and CPU at a specific resolution
def process_frame(frame, resolution):
    frame_resized = cv2.resize(frame, resolution)

    start_gpu = time.time()
    gpu_results = gpu_reader.readtext(frame_resized)
    end_gpu = time.time()

    start_cpu = time.time()
    cpu_results = cpu_reader.readtext(frame_resized)
    end_cpu = time.time()

    return gpu_results, cpu_results, end_gpu - start_gpu, end_cpu - start_cpu

# Function to process video and evaluate performance for different resolutions
def process_video(video_path, skip_interval=2):
    cap = cv2.VideoCapture(video_path)

    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    frame_count = 0
    results_per_resolution = {resolution: {'gpu_times': [], 'cpu_times': [], 'frames_processed': 0} for resolution in resolutions}

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % skip_interval == 0:
            for resolution in resolutions:
                gpu_results, cpu_results, gpu_time, cpu_time = process_frame(frame, resolution)

                # Store results
                results_per_resolution[resolution]['gpu_times'].append(gpu_time)
                results_per_resolution[resolution]['cpu_times'].append(cpu_time)
                results_per_resolution[resolution]['frames_processed'] += 1

                # Optionally draw results on the frame
                # (similar drawing logic as before)

        frame_count += 1

    cap.release()
    cv2.destroyAllWindows()

    # Evaluate performance for each resolution
    for resolution, data in results_per_resolution.items():
        avg_gpu_time = np.mean(data['gpu_times']) if data['gpu_times'] else 0
        avg_cpu_time = np.mean(data['cpu_times']) if data['cpu_times'] else 0
        fps_gpu = 1 / avg_gpu_time if avg_gpu_time > 0 else 0
        fps_cpu = 1 / avg_cpu_time if avg_cpu_time > 0 else 0

        print(f"Resolution {resolution}: Average FPS for GPU: {fps_gpu:.2f}, Average FPS for CPU: {fps_cpu:.2f}")

# Specify the path to your video file and frame skipping interval
frame_skip_interval = 10
process_video(video_file_path, skip_interval=frame_skip_interval)

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

Resolution (640, 480): Average FPS for GPU: 0.15, Average FPS for CPU: 0.15
Resolution (800, 600): Average FPS for GPU: 0.12, Average FPS for CPU: 0.11
Resolution (1024, 768): Average FPS for GPU: 0.07, Average FPS for CPU: 0.07
Resolution (1280, 720): Average FPS for GPU: 0.05, Average FPS for CPU: 0.05
